<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/models/3DModels/LandmarkMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, mlflow setup and unzip

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install -q mlflow
!databricks configure --host https://community.cloud.databricks.com/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
Username: gfesta24@gmail.com
Password: 
Repeat for confirmation: 


In [ ]:
import sys
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid
import torch.optim as optim
import numpy as np

Here we set the paths, so change them to load the Landmarks. You can mine the landmark with Landmark Extraction file in our 3DTransformations folder

In [ ]:
train_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/training_set.csv")
test_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/test_set.csv")

# Data Loading

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 1024

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X_array, Y_array, transform=None):
        self.X = X_array
        self.Y = Y_array
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index]

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

This is needed to extract the np.array from the csv

In [ ]:
test_df['landmarks'] = test_df['landmarks'].apply(lambda lab: eval(lab))

train_df['landmarks'] = train_df['landmarks'].apply(lambda lab: eval(lab))
print(train_df)

In [ ]:
print(type(train_df.at[0,"landmarks"]))

label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}

test_df['label'] = test_df['label'].apply(lambda lab: label_dict[lab])

train_df['label'] = train_df['label'].apply(lambda lab: label_dict[lab])

array_train = train_df['landmarks'].to_numpy()
X_train = np.stack([np.array(lst) for lst in array_train])
y_train = train_df['label'].to_numpy()
array_test = test_df['landmarks'].to_numpy()
X_test = np.stack([np.array(lst) for lst in array_test])
y_test = test_df['label'].to_numpy()

<class 'list'>


In [ ]:
train_dataset = CustomDataset(X_train, y_train, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = CustomDataset(X_test, y_test, transform=transforms.ToTensor())

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print( X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(26705, 478, 3) (26705,) (6678, 478, 3) (6678,)


# Model

In [ ]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

In [ ]:
class MultiClassificator(nn.Module):
  def __init__(self, in_size: int, hidden_size: int, num_classes: int):
    super(MultiClassificator, self).__init__()

    self.fc1 = nn.Linear(in_size, 1024, dtype=torch.float64)
    self.fc2 = nn.Linear(1024, 500, dtype=torch.float64)
    self.fc3 = nn.Linear(500, 100, dtype=torch.float64)
    self.fc4 = nn.Linear(100, num_classes, dtype=torch.float64)

  def forward(self, x: torch.Tensor):
    b = x.shape[0]
    x = x.view(b,-1)

    out = self.fc1(x)
    out = F.relu(out)
    out = self.fc2(out)
    out = F.relu(out)
    out = self.fc3(out)
    out = F.relu(out)
    out = self.fc4(out)
    return out

# Training loop

In [ ]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/gfesta24@gmail.com/LandmarkMLP")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3698710157086970', creation_time=1687350733677, experiment_id='3698710157086970', last_update_time=1687352897667, lifecycle_stage='active', name='/Users/gfesta24@gmail.com/LandmarkMLP', tags={'mlflow.experiment.sourceName': '/Users/gfesta24@gmail.com/LandmarkMLP',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'gfesta24@gmail.com',
 'mlflow.ownerId': '1923923806180228'}>

In [ ]:
param_grid = {'batch_size': [64, 128, 256, 512],'lr': [0.0001, 0.001], "decay":[0, 0.001, 0.01]}
expanded_grid = ParameterGrid(param_grid)
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("/Users/gfesta24@gmail.com/LandmarkMLP")


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for i in range(len(expanded_grid)):

  runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], filter_string=" and ".join([f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()]))
  if not len(runs) == 0:
    print("RUN: ", [f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()], " già completata" )
  else:
    train_loader = DataLoader(train_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=True)

    test_loader = DataLoader(test_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=False)

    model = MultiClassificator(478*3, 1024,  7)
    optimizer = optim.Adam(model.parameters(), lr=expanded_grid[i]['lr'], weight_decay=expanded_grid[i]['decay'])
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    mlflow.start_run()

    mlflow.set_tag("model_name", "LandmarkMLP")
    mlflow.log_param("lr", expanded_grid[i]['lr'])
    mlflow.log_param("batch_size", expanded_grid[i]['batch_size'])
    mlflow.log_param("decay", expanded_grid[i]['decay'])

    acc_list_train=[]
    acc_list_test=[]
    num_epochs = 100
    best_loss = 1000
    patience=3
    counter = 0
    stop = False

    # Training loop
    best_accuracy = 0.0


    for epoch in range(num_epochs):
            model.train()
            print(counter)
            if stop:
              print(stop)
              break
            running_loss = 0.0
            running_acc = 0
            seen = 0
            for images, labels in train_loader:

              images = images.to(device)
              labels = labels.to(device)


              outputs = model(images)

              loss = criterion(outputs, labels)

              _, acc = accuracy(outputs, labels)
              seen +=labels.shape[0]

              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              running_loss += loss.item()
              running_acc += acc

            print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/seen:.4f}')
            acc_list_train.append(running_acc/len(train_loader))
            mlflow.log_metric("train_loss", running_loss / len(train_loader), step=epoch)
            mlflow.log_metric("train_acc", running_acc/seen, step=epoch)
            model.eval()

            tot_corrette = 0
            tot_eseguite = 0
            running_test_loss = 0
            val_loss = 0

            with torch.no_grad():

              for images, labels in test_loader:
                  images = images.to(device)
                  labels = labels.to(device)

                  outputs = model(images)
                  test_loss = criterion(outputs, labels)
                  _, n_corrette=accuracy(outputs, labels)

                  running_test_loss += test_loss.item()
                  tot_corrette+=n_corrette.item()
                  tot_eseguite+=labels.shape[0]

              test_acc=100* (tot_corrette/tot_eseguite)
              val_loss = running_test_loss / len(test_loader)
              acc_list_test.append(test_acc)
              print("Test acc: ", test_acc)
              print("Test loss: ", val_loss)
              mlflow.log_metric("test_acc", test_acc, step=epoch)
              mlflow.log_metric("test_loss", val_loss, step=epoch)


            if val_loss < best_loss:
              print("MIGLIORATO")
              torch.save(model.state_dict(), 'model_weights.pth')
              best_loss = val_loss
              best_model_train_acc=running_acc/seen
              best_model_test_acc=test_acc
              best_model_test_loss=val_loss
              best_model_train_loss=running_loss / len(train_loader)
              counter = 0
              # Salva i pesi del modello se la validation loss è migliorata
              torch.save(model.state_dict(), 'best_model.pt')
            else:
              counter += 1
            # Verifica se raggiunto il criterio di early stopping
              if counter >= patience:
                  print(f'Early stopping at epoch {epoch+1}')
                  mlflow.set_tag("Epochs_stopped", epoch+1)
                  mlflow.log_artifact("best_model.pt")
                  mlflow.log_metric("best_test_acc", best_model_test_acc)
                  mlflow.log_metric("best_test_loss", best_model_test_loss)
                  mlflow.log_metric("best_train_acc", best_model_train_acc)
                  mlflow.log_metric("best_train_loss", best_model_train_loss)
                  mlflow.end_run()
                  stop=True
            print("BEST TEST LOSS: ", best_loss)

RUN:  ["params.lr = '0.0001'", "params.decay = '0'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.001'", "params.decay = '0'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.0001'", "params.decay = '0.001'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.001'", "params.decay = '0.001'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.0001'", "params.decay = '0.01'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.001'", "params.decay = '0.01'", "params.batch_size = '64'"]  già completata
RUN:  ["params.lr = '0.0001'", "params.decay = '0'", "params.batch_size = '128'"]  già completata
RUN:  ["params.lr = '0.001'", "params.decay = '0'", "params.batch_size = '128'"]  già completata
RUN:  ["params.lr = '0.0001'", "params.decay = '0.001'", "params.batch_size = '128'"]  già completata
RUN:  ["params.lr = '0.001'", "params.decay = '0.001'", "params.batch_size = '128'"]  già completata
RUN:  ["p

# Testing

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, classification_report

model = MultiClassificator(478*3, 1024,  7)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()
model.to(device)



# Calcola le metriche sul test dataset
model.eval()  # Imposta il modello in modalità di valutazione (non addestramento)
test_predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(torch.nn.functional.softmax(outputs, dim=1), 1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average=None)
f1 = f1_score(test_labels, test_predictions, average=None)
#auc_roc = roc_auc_score(test_labels, test_predictions, multi_class='ovr')
classification_rep = classification_report(test_labels, test_predictions)

print("Test Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)
#print("AUC-ROC:", auc_roc)
print("Classification Report:\n", classification_rep)